In [5]:
!pip install mindspore
!pip install lda
!pip install pandas
!pip install numpy
!pip install nltk
!pip install scikit-learn
!pip install scipy
!pip install torch
!pip install gensim
!pip install regex

     |████████████████████████████████| 158.8 MB 53 bytes/s a 0:00:011
     |████████████████████████████████| 348 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 1.5 MB 5.5 MB/s eta 0:00:01


In [3]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [6]:
import string
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wn = WordNetLemmatizer()

import os
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# Load the data
train_data = pd.read_csv("arp_train.csv")
test_data = pd.read_csv("arp_test.csv")

# Remove any rows with missing values
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [10]:
def clean_text(lines, review_lines):
    for line in lines:
        tokens = word_tokenize(line)
        #convert to lower case
        tokens = [w.lower() for w in tokens]
        #remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        #remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        #filter out stop words
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        words = [wn.lemmatize(w) for w in words]
        review_lines.append(words)
    return review_lines

In [ ]:
clean_test = list()
clean_text(test_data["review"], clean_test)
clean_train = list()
clean_text(train_data["review"], clean_train)

train_docs = [' '.join(sublist) for sublist in clean_train]
test_docs = [' '.join(sublist) for sublist in clean_test]
print("done with cleaning data")

In [18]:
train_labels = train_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

test_labels = test_data['label'].apply(lambda x: 1 if x == 'pos' else 0)


In [ ]:

vectorizer = CountVectorizer()
train_bow = vectorizer.fit_transform(train_docs)
test_bow = vectorizer.transform(test_docs)


In [ ]:
print("hey")
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(train_bow)
print(1)
#Generate sparse vectors (word embeddings) for the train, validation and test sets
train_embeddings = lda.transform(train_bow)
print(2)
test_embeddings = lda.transform(test_bow)

In [ ]:

from sklearn.model_selection import train_test_split

lr = LogisticRegression(random_state=42)
lr.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = lr.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))

In [ ]:
from sklearn.svm import SVC
# Initialize an SVM classifier
svm = SVC(random_state=42)

# Fit the classifier on the training set
svm.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = svm.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))


In [11]:
# Load the data
train_data = pd.read_csv("./lmr_train_mixed_labels.csv")
test_data = pd.read_csv("lmr_test.csv")

# Remove any rows with missing values
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

train_dirt = train_data['review']
test_dirt = test_data['review']

train = list()
test = list()

train = clean_text(train_dirt, train)
test = clean_text(test_dirt, test)

train_data['tokens'] = train
test_data['tokens'] = test

In [12]:
train_docs = [' '.join(sublist) for sublist in train]
test_docs = [' '.join(sublist) for sublist in test]

In [ ]:
print("Started Vectorization")
# vectorize text data
vectorizer = CountVectorizer()
train_bow = vectorizer.fit_transform(train)
test_bow = vectorizer.transform(test)


lda = LatentDirichletAllocation(n_components=50, random_state=42)
lda.fit(train_bow)
print(1)
# Generate sparse vectors (word embeddings) for the train, validation and test sets
train_embeddings = lda.transform(train_bow)
print(2)
test_embeddings = lda.transform(test_bow)



In [ ]:
train_labels = train_data['label']
test_labels = test_data['label']

from sklearn.svm import SVC

# Initialize an SVM classifier
svm = SVC(random_state=42)

# Fit the classifier on the training set
svm.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = svm.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))

from sklearn.model_selection import train_test_split

lr = LogisticRegression(random_state=42)
lr.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = lr.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))

In [13]:
import gensim
tagged_documents = []
for i, doc in enumerate(train_docs):
    tokens = gensim.utils.simple_preprocess(doc)
    tagged_documents.append(TaggedDocument(tokens, [i]))

In [14]:
model = Doc2Vec(tagged_documents, vector_size=400, window=5, min_count=1, workers=4)


In [15]:

train_p_vectors = []
for sentence in train_docs:
    # Infer a new vector for the sentence
    new_vector = model.infer_vector(sentence.split())
    train_p_vectors.append(new_vector)


test_p_vectors = []
for sentence in test_docs:
    # Infer a new vector for the sentence
    new_vector = model.infer_vector(sentence.split())
    test_p_vectors.append(new_vector)



In [16]:

train_embeddings = np.array(train_p_vectors)
print(train_embeddings.shape)
test_embeddings = np.array(test_p_vectors)
print(test_embeddings.shape)

(50092, 400)
(25060, 400)


In [ ]:
train_labels = train_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

test_labels = test_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

In [19]:
from sklearn.model_selection import train_test_split

lr = LogisticRegression(random_state=42)
lr.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = lr.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))

Validation set results:
Accuracy: 0.8074
F1 Score: 0.8073
Precision: 0.8079
Recall: 0.8074


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
train_labels = train_data['label']
test_labels = test_data['label']

from sklearn.svm import SVC

# Initialize an SVM classifier
svm = SVC(random_state=42)

# Fit the classifier on the training set
svm.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = svm.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))



Validation set results:
Accuracy: 0.8076
F1 Score: 0.8076
Precision: 0.8077
Recall: 0.8076


In [ ]:
# Load the data
train_data = pd.read_csv("arp_train.csv")
test_data = pd.read_csv("arp_test.csv")

# Remove any rows with missing values
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [ ]:
clean_test = list()
clean_text(test_data["review"], clean_test)
clean_train = list()
clean_text(train_data["review"], clean_train)

train_docs = [' '.join(sublist) for sublist in clean_train]
test_docs = [' '.join(sublist) for sublist in clean_test]
print("done with cleaning data")

In [ ]:
train_labels = train_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

test_labels = test_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

In [ ]:
import gensim
tagged_documents = []
for i, doc in enumerate(train_docs):
    tokens = gensim.utils.simple_preprocess(doc)
    tagged_documents.append(TaggedDocument(tokens, [i]))

In [ ]:
model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
train_p_vectors = []
for sentence in train_docs:
    # Infer a new vector for the sentence
    new_vector = model.infer_vector(sentence.split())
    train_p_vectors.append(new_vector)


test_p_vectors = []
for sentence in test_docs:
    # Infer a new vector for the sentence
    new_vector = model.infer_vector(sentence.split())
    test_p_vectors.append(new_vector)

In [ ]:
train_labels = train_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

test_labels = test_data['label'].apply(lambda x: 1 if x == 'pos' else 0)

In [ ]:
train_embeddings = np.array(train_p_vectors)
print(train_embeddings.shape)
test_embeddings = np.array(test_p_vectors)
print(test_embeddings.shape)

In [ ]:
from sklearn.model_selection import train_test_split

lr = LogisticRegression(random_state=42)
lr.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = lr.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))

In [20]:
from sklearn.linear_model import SGDClassifier

# Initialize an SVM classifier with SGD algorithm
svm = SGDClassifier(loss='hinge', random_state=42)

# Fit the classifier on the training set
svm.fit(train_embeddings, train_labels)

# Evaluate the model on the validation set
val_preds = svm.predict(test_embeddings)
val_acc = accuracy_score(test_labels, val_preds)
val_f1 = f1_score(test_labels, val_preds, average='weighted')
val_precision = precision_score(test_labels, val_preds, average='weighted')
val_recall = recall_score(test_labels, val_preds, average='weighted')

print("Validation set results:")
print("Accuracy: {:.4f}".format(val_acc))
print("F1 Score: {:.4f}".format(val_f1))
print("Precision: {:.4f}".format(val_precision))
print("Recall: {:.4f}".format(val_recall))


Validation set results:
Accuracy: 0.8035
F1 Score: 0.8030
Precision: 0.8066
Recall: 0.8035
